# Biconvex Lens Parabola

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchlensmaker as tlm


shape = tlm.Parabola(height=15., a=nn.Parameter(torch.tensor(0.005)))

lens = tlm.SymmetricLens(shape, (1.0, 1.49), outer_thickness=0.5)

optics = tlm.OpticalSequence(
    tlm.PointSourceAtInfinity(beam_diameter=20),
    tlm.Gap(10.),
    lens,
    tlm.Gap(45.0),
    tlm.FocalPoint(),
)

tlm.render_plt(optics)

tlm.optimize(
    optics,
    optimizer = optim.Adam(optics.parameters(), lr=1e-3),
    sampling = {"rays": 10},
    num_iter = 100
)

tlm.render_plt(optics)

In [ ]:
from IPython.display import display

part = tlm.lens_to_part(lens)
display(part)

print("Outer thickness:", lens.outer_thickness().item())
print("Inner thickness:", lens.inner_thickness().item())

# bd.export_step(part, "lens.step")